**Question 1: 
Bellman Optimality Equations
Using a contraction argument, show that there exists a solution to the Bellman optimality equations. That is : show that the Bellman optimality operator is a contraction mapping. (Doina covered the linear case in class ; here you need to go through the same steps but in the nonlinear case).**


We can show that the Bellman Optimality Equations converge to the optimal solution by showing that the
the Bellman Optimality Backup is a contraction.  The distance between functions u and v is
measured by the max norm, which describes the largest difference between state values: 


\begin{align}
{||u-v||}_\infty = \mathop{max}_{s \mathop{\in} S} \mathop{|u(s)-v(s)|}
\end{align}

The Bellman expectation backup is a contraction defined as $$T^\pi(v) = R^\pi + \mathop{\gamma}P^{\pi}v$$
It is a $\gamma$-contraction which makes value functions closer by at least $\gamma$


The contraction mapping theorem states that for any metric space $V$ that is complete under an operator $T(v)$ where T is a $\gamma$-contraction, $T$ converges to a unique fixed point at a linear convergence rate of $\gamma$. Therefore, the Bellman expectation operator $T^{\pi}$ has a unique fixed point which contains $v_{\pi}$. Iterative policy evaluation converges on $v_{\pi}$. 

The Bellman Optimality backup is a contraction defined as

\begin{align}
T^*(v) = \mathop{max}_{a \in A} R^a + \gamma P^a v
\end{align}
It is a $\gamma$-contraction which makes the functions closer by at least $\gamma$

\begin{align}
{||T^{*}(u)-T^{*}(v)||}_\infty & = {||\mathop{max}_{a \in A}(R^a+{\gamma}P^{a}u) - \mathop{max}_{a \in A}(R^a+{\gamma}P^{a}v)
||}_\infty \\
&= || \space{ {||(R^a+{\gamma}P^{a}u)||}_\infty - {||(R^a+{\gamma}P^{a}v)
||}_\infty||}_\infty \\
&= || \space||{{(R^a+{\gamma}P^{a}u) - (R^a+{\gamma}P^{a}v)
||}_\infty||}_\infty \\
&= || \space||{{{\gamma}P^{a}u     - {\gamma}P^{a}v||}_\infty||}_\infty \\
&\leq ||{{\gamma}P^{a}\space{||u-v||}_\infty||}_\infty \\
&\leq {\gamma}\space{||u-v||}_\infty\\
\end{align}


**Problem 3.1
Implement and compare empirically the performance of value iteration, policy iteration and modified policy iteration. Modified policy iteration is a simple variant of policy iteration in which the evaluation step is only partial. You can consult the Puterman (1994) textbook for more information. You should first implement the algorithms in a 2-states MDP specified as follows:

Transition Probabilities: 

    P(s_0 | s_0, a_0) = 0.5, 
    P(s_1 | s_0, a_0) = 0.5, 
    P(s_0 | s_0, a_1) = 0,
    P(s_1 | s_0, a_1) = 1, 
    P(s_1 | s_0, a_2) = 0, 
    P(s_1 | s_1, a_2) = 1
Rewards: 

    r(s_0, a_0) = 5, r(s_0, a_1) = 10, r(s_1, a_2) = -1

**

In [215]:
# build mdp
import numpy as np
num_states = 2
num_actions = 3
transitions = np.zeros((num_states, num_states, num_actions,), np.float32)
rewards = np.zeros((num_states, num_states, num_actions), np.int16)
# according to assn given
# transitions[s,s1,a]  
transitions[0,0,0] = 0.5
transitions[0,1,0] = 0.5
transitions[0,1,1] = 1.0
transitions[1,1,2] = 1.0


rewards[0,0,0] = 5
rewards[0,1,0] = 5
rewards[0,1,1] = 10
rewards[1,1,2] = -1



In [251]:
#### value iteration

def policy_evaluation(policy, ts, rs, gamma):
    num_states = ts.shape[0]
    num_actions = ts.shape[2]
    V = np.zeros(num_states)
    theta = .0001
    
    while True:
        e = 0
        for s in range(num_states):
            v = 0
            # look at possible actions from this state
            for a in range(num_actions):
                for s1 in range(num_states):
                    prob = ts[s,s1,a]
                    if prob > 0:
                        r = rs[s,s1,a]
                        v+=prob * (r+gamma*V[s1])
            e = max(delta,np.abs(v-V[s]))
            V[s] = v
            
        if e < theta:
            break
        return V
            
def policy_improvement(ts, rs, gamma):
    num_states = ts.shape[0]
    num_actions = ts.shape[2]
    p = np.ones([num_states,num_actions])/float(num_actions)
    while True:
        V = policy_evaluation(p,ts,rs,gamma)
        # flag to catch when policy changes
        stable = True
        for s in range(num_states):
            policy_action = np.argmax(p[s])
            #av = np.zeros(num_actions)
            valid_actions = []
            valid_values = []
            for a in range(num_actions):
                av = 0
                is_valid = False
                for s1 in range(num_states):
                    prob = ts[s,s1,a]
                    if prob > 0:
                        r = rs[s,s1,a]
                        av += prob*(r+gamma*V[s1])
                        is_valid = True
                if is_valid:
                    valid_actions.append(a)
                    valid_values.append(av)
            best_action = valid_actions[np.argmax(valid_values)]
        
            if policy_action != best_action:
                stable = False
            p[s] = np.eye(num_actions)[best_action]
            
        if stable:
            return p, V
    
    
policy = [0,2]
policy, V = policy_improvement(transitions,rewards,.95)

In [260]:

def get_action_value(s,ts,rs,gamma):
    # find value for actions
    valid_actions = []
    valid_values = []
    for a in range(num_actions):
        aval = 0
        is_valid = False
        for s1 in range(num_states):
            prob = ts[s,s1,a]
            if prob > 0:
                r = rs[s,s1,a]
                aval +=prob * (r+gamma*V[s1])
                is_valid = True

        if is_valid:
            valid_actions.append(a)
            valid_values.append(aval)

    best_action_value = np.max(valid_values)  
    best_action = valid_actions[np.argmax(valid_values)]
    return best_action_value, best_action



def value_iteration(ts,rs,gamma):
    num_states = ts.shape[0]
    num_actions = ts.shape[2]
    V = np.zeros(num_states)
    theta = .0001
    while True: 
        e = 0
        for s in range(num_states):
            best_action_value, best_action = get_action_value(s,ts,rs,gamma)
            e = max(e,np.abs(best_action_value-V[s]))
            V[s] = best_action_value
            print(s,best_action,best_action_value,e)
        if e<theta:
            break
        policy = np.zeros([num_states, num_actions])

        
    for s in range(num_states):
        best_action_value, best_action = get_action_value(s,ts,rs,gamma)
        policy[s,best_action] = 1.0
    return policy,V
            
pol,v=value_iteration(transitions,rewards,.95)         



(0, 0, 11.65, 11.65)
(1, 2, -1.95, 11.65)
(0, 0, 11.65, 0)
(1, 2, -1.95, 0)
(array([[ 1.,  0.,  0.],
       [ 0.,  0.,  1.]]), array([ 11.65,  -1.95]))
